In [ ]:
import requests
from pyspark.sql.functions import col, max as spark_max, count, explode, sum as sum_spark, collect_list, desc, first, round, to_timestamp, current_date, datediff, udf, when
from pyspark.sql.types import DoubleType

In [ ]:
user = spark.table("steam.silver.user_games")
details = spark.table("steam.silver.details_games")

In [ ]:
df_unificado = user.join(details, user.appid == details.steam_appid, "left").drop(details.steam_appid)

In [ ]:
df = df_unificado.select('appid', 'name', 'type', 'playtime_forever', 'rtime_last_played', 
                         'header_image', 'is_free', 'publishers', 'required_age', 'website', 'short_description', 
                         'id_genres', 'description_genres', 'id_categories', 'description_categories', 'languages',
                           'OS', 'Processor', 'Memory', 'Graphics', 'Storage', 'Currency', 'Price')

In [ ]:
%sql
-- Usando SQL mágico do Databricks para criar schema se não existir
CREATE SCHEMA IF NOT EXISTS hive_metastore.gold

In [ ]:
df_dim = df.select('appid', 'name', 'type', 'header_image', 'publishers', 'required_age',
                    'website', 'short_description', 'languages', 'OS', 'Processor', 'Memory',
                      'Graphics', 'Storage').distinct()

# Escrita na camada gold
df_dim.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("hive_metastore.gold.games")

In [ ]:
df_prices = df.select('appid', 'Currency', 'Price', 'is_free').distinct()

# Pegar as moedas únicas da coluna Currency, excluindo '-'
unique_currencies = [row.Currency for row in df.select("Currency").distinct().collect() if row.Currency != '-']

# Busca as cotações atuais
def get_exchange_rate(currency_code):
    url = f"https://economia.awesomeapi.com.br/json/last/{currency_code}-BRL"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        pair_key = currency_code + 'BRL'
        return float(data[pair_key]['bid'])
    return 1.0

# Busca as taxas para todas as moedas capturadas
rates = {currency: get_exchange_rate(currency) for currency in unique_currencies}
rates['-'] = 1.0  # Trate '-' como BRL, sem conversão

# UDF PySpark para converter preços
def convert_price(currency, price, is_free):
    if is_free:
        return 0.0
    rate = rates.get(currency, 1.0)
    return price * rate

convert_price_udf = udf(convert_price, DoubleType())

# Aplica a conversão criando coluna nova
df_converted = df_prices.withColumn(
    "PriceBRL",
    round(convert_price_udf(col("Currency"), col("Price"), col("is_free")), 2)
)

df_updated = df_converted.withColumn(
    "is_free",
    when(col("Price") == 0, True).otherwise(False)
)

# Escrita na camada gold
df_updated.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("hive_metastore.gold.prices")

In [ ]:
# Converter para timestamp
df_playtime= df.select("appid", "playtime_forever", "rtime_last_played").distinct()

# Encontrar a última data por appid
df_max_dates = (
    df_playtime.groupBy("appid")
         .agg(spark_max("rtime_last_played").alias("ultima_data_jogada"))
)

# Filtrar o DataFrame original para manter apenas os registros da última data
df_filtrado = (
    df_playtime.join(df_max_dates,
               (df_playtime.appid == df_max_dates.appid) &
               (df_playtime.rtime_last_played == df_max_dates.ultima_data_jogada),
               "inner"
    )
    .select(df_playtime["*"])  # mantém todas as colunas originais
).orderBy(col("playtime_forever").desc())

df_com_tempo_sem_jogar = (
    df_filtrado.withColumn(
        "time_without_play",
        datediff(current_date(), col("rtime_last_played")).cast("int")
    )
)

# Calcular o total de horas jogadas (variável Python)
total_horas = df_com_tempo_sem_jogar.agg(sum_spark("playtime_forever").alias("total_horas")).collect()[0]["total_horas"]

# Adicionar coluna com a proporção de tempo jogado por appid
df_proporcao = df_com_tempo_sem_jogar.withColumn(
    "proporcao_tempo",
    round((col("playtime_forever") / total_horas) * 100, 2)
)


# Escrita na camada gold
df_proporcao.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("hive_metastore.gold.playtime")

In [ ]:
# Seleciona as colunas 'appid', 'id_genres' e 'description_genres' do DataFrame original 'df' e remove duplicatas
genres = df.select('appid', 'id_genres', 'description_genres').distinct()

# Renomeia a coluna 'description_genres' para 'genres' para facilitar operações futuras
genres = genres.withColumnRenamed("description_genres", "genres")

# Agrupa por gênero, conta quantidade de appids e gera lista dos appids associados para cada gênero
df_generos_por_app = (
    genres.groupBy("genres")
        .agg(
            count("appid").alias("qtd_games"),
            collect_list("appid").alias("appids")
        )
        .orderBy(desc("qtd_games"))
)

# Explode a coluna appids para que cada appid fique em uma linha única com seu gênero
df_exploded = df_generos_por_app.select("genres", "qtd_games", explode("appids").alias("appid"))

# Realiza o join com o DataFrame que possui playtime_forever usando a coluna appid
df_joined = df_exploded.join(df_proporcao.select("appid", "playtime_forever"), on="appid", how="left")

# Agrupa por gênero incluindo qtd_games via função first()
df_generos_por_app_join = (
    df_joined.groupBy("genres")
        .agg(
            first("qtd_games").alias("qtd_games"),
            round(sum_spark("playtime_forever"), 2).alias("playtime_total"),
            collect_list("appid").alias("appids")
        )
        .orderBy(desc("playtime_total"))
)
    

# Escrita na camada gold
df_generos_por_app_join.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("hive_metastore.gold.genres")


In [ ]:
# Seleciona as colunas 'appid', 'id_categories' e 'description_categories' do DataFrame original 'df' e remove duplicatas

'id_categories', 'description_categories'

categories = df.select('appid', 'id_categories', 'description_categories').distinct()

# Renomeia a coluna 'description_categories' para 'categories' para facilitar operações futuras
categories = categories.withColumnRenamed("description_categories", "categories")

# Agrupa por gênero, conta quantidade de appids e gera lista dos appids associados para cada gênero
df_categories_por_app = (
    categories.groupBy("categories")
        .agg(
            count("appid").alias("qtd_games"),
            collect_list("appid").alias("appids")
        )
        .orderBy(desc("qtd_games"))
)

# Explode a coluna appids para que cada appid fique em uma linha única com seu gênero
df_exploded = df_categories_por_app.select("categories", "qtd_games", explode("appids").alias("appid"))

# Realiza o join com o DataFrame que possui playtime_forever usando a coluna appid
df_joined = df_exploded.join(df_proporcao.select("appid", "playtime_forever"), on="appid", how="left")

# Agrupa por gênero incluindo qtd_games via função first()
df_categories_por_app_join = (
    df_joined.groupBy("categories")
        .agg(
            first("qtd_games").alias("qtd_games"),
            round(sum_spark("playtime_forever"), 2).alias("playtime_total"),
            collect_list("appid").alias("appids")
        )
        .orderBy(desc("playtime_total"))
)

# Escrita na camada gold
df_categories_por_app_join.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("hive_metastore.gold.categories")